## Pickle

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
test_dataset  = datasets.MNIST(root='data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=1000)

# 3. Define Neural Network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # Define a neural network model using nn.Sequential for a simple feedforward network
        self.model = nn.Sequential(
            nn.Flatten(),              # Flattens the 28x28 input image into a 1D tensor of size 784
            nn.Linear(28*28, 128),     # Fully connected layer from 784 input features to 128 output features
            nn.ReLU(),                 # ReLU activation function introduces non-linearity. Max(0, x)
            nn.Linear(128, 10)         # Fully connected layer from 128 features to 10 classes (for classification)
        )
        
    def forward(self, x):
        return self.model(x)

In [ ]:
## Without loading

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model = SimpleNN()
loaded_model.to(device)

correct = 0
total = 0

with torch.no_grad(): # Disables gradient tracking. Reduces memory usage and speeds up computation during inference, since we don’t need gradients.
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device) # Moves the images and labels to the same device as the model (CPU or GPU).
        outputs = loaded_model(images) # Feeds batch of 1000 images through the model to get predictions.
        _, predicted = torch.max(outputs.data, 1) # returns both the max value and its index across class dimension.
        print(len(predicted))
        total += labels.size(0) # Adds the number of labels in the batch to the total number of samples.
        correct += (predicted == labels).sum().item() # Compares predicted labels to true labels. .sum() counts the number of correct predictions in the batch. .item() converts the tensor to a Python integer and adds it to the running total.

print(f"Test Accuracy: {100 * correct / total:.2f}%")
# loaded_model.eval()

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
Test Accuracy: 11.45%


torch.Size([1000, 10])

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model = SimpleNN()
loaded_model.load_state_dict(torch.load("model_weights.pth", map_location=device))
loaded_model.to(device)

correct = 0
total = 0

with torch.no_grad(): # Disables gradient tracking. Reduces memory usage and speeds up computation during inference, since we don’t need gradients.
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device) # Moves the images and labels to the same device as the model (CPU or GPU).
        outputs = loaded_model(images) # Feeds batch of 1000 images through the model to get predictions.
        _, predicted = torch.max(outputs.data, 1) # returns both the max value and its index across class dimension.
        print(outputs.shape)
        print(len(predicted))
        total += labels.size(0) # Adds the number of labels in the batch to the total number of samples.
        correct += (predicted == labels).sum().item() # Compares predicted labels to true labels. .sum() counts the number of correct predictions in the batch. .item() converts the tensor to a Python integer and adds it to the running total.

print(f"Test Accuracy: {100 * correct / total:.2f}%")
# loaded_model.eval()

torch.Size([1000, 10])
1000
torch.Size([1000, 10])
1000
torch.Size([1000, 10])
1000
torch.Size([1000, 10])
1000
torch.Size([1000, 10])
1000
torch.Size([1000, 10])
1000
torch.Size([1000, 10])
1000
torch.Size([1000, 10])
1000
torch.Size([1000, 10])
1000
torch.Size([1000, 10])
1000
Test Accuracy: 97.45%


## ONNX

In [5]:
import onnxruntime as ort
import numpy as np
import torch

# Load the ONNX model
session = ort.InferenceSession("model.onnx")

# Prepare input data (NumPy array, same shape as during training)
# For MNIST: batch of 1 image, 1 channel, 28x28
dummy_image = torch.randn(1, 1, 28, 28)
input_array = dummy_image.numpy().astype(np.float32)

# Get the input name for the model (matches the name used in export)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# Run inference
outputs = session.run([output_name], {input_name: input_array})

# Get predicted class
predicted_class = np.argmax(outputs[0], axis=1)
print(f"Predicted class: {predicted_class}")

Predicted class: [5]


In [ ]:
outputs[0]

array([[-15.186843 , -17.99852  ,  -9.234358 ,   0.6698692,  -1.6174479,
          3.9458423, -19.283533 ,  -4.363623 ,   1.85484  ,   1.5067648]],
      dtype=float32)

In [ ]:
## Don't need softmax. Softmax turns predicted vals into probabilities.
# Needs softmax when we want to show prediction confidence, need threshold confidence, visualizing model uncertainty